# Android WeatherApp Project Guide for absolute beginners - 3. MainActivity.java

I created this guide as a reference for my personal learning on Android app development. The original tutorial available in link in the references below.


**References**  
- GeeksforGeeks (2021) Making Weather App in Android Studio | Android Projects | GeeksforGeeks https://www.youtube.com/watch?v=q7NF-2gtfEU&t=864s

In [ ]:
from google.colab import drive
drive.mount('here')

In [ ]:
import cv2 
from google.colab.patches import cv2_imshow

def show_img(path):
    img = cv2.imread(path)
    cv2_imshow(img)

# MainActivity.java





## 1) Declare API variables


## 2) Initialise them in `onCreate`
inside the **`protected void onCreate(Bundle savedInstanceState){}`** method

### `setAdapter()`

- Excerpt from StackOverFlow:


    mDrawerList.setAdapter(new ArrayAdapter<String>
    (this,R.layout.drawer_list_item, mServices));


A list adapter is an object that adapts a collection objects for display in a ListView. ArrayAdapter is one simple implementation that maps an array of objects.

This line is mapping an array of strings (mServices) for display in a ListView (mDrawerList). The second argument to the adapter's constructor is the layout that will be used to render each list item.

- https://stackoverflow.com/questions/33703548/how-to-use-setadapter

In [ ]:
# MainActivity.java

public class MainActivity extends AppCompatActivity {

    # Declare variables with there id's.
    private RelativeLayout homeRL;

    ...

    private ImageView backIV, iconIV, searchIV;

    # Add WeatherRVModal variable here.
    # then initialise these down below in onCreate too.
    private ArrayList<WeatherRVModal> weatherRVModalArrayList;
    private WeatherRVAdapter weatherRVAdapter;  # adaptor class

    @Override
    protected void onCreate(Bundle savedInstanceState) {
        super.onCreate(savedInstanceState);
        setContentView(R.layout.activity_main);

        # initialise the variables here.
        homeRL = findViewById(R.id.idRLHome);

        ...

        searchIV = findViewById(R.id.idIVSearch);
            
        # API we will going to use.
        weatherRVModalArrayList = new ArrayList<>();
        # Init ialise the adapter.
        weatherRVAdapter = new WeatherRVAdapter(this, weatherRVModalArrayList);
        # Set this adapter to the RecyclerView.
        weatherRV.setAdapter(weatherRVAdapter);

        # Inisialise the LocationManager.
        locationManager = (LocationManager) getSystemService(Context.LOCATION_SERVICE);
    }

...

## 3) Get weather info

In [ ]:
...

    # Create a method for weather info,
    # and another for user location.
    private  void getWeatherInfo(String cityName){

        # Parse the data from API.
        # so first, create a url.
        String url = "http://api.weatherapi.com/v1/forecast.json?key=34a7fe8762854dd7b8765811212509&q=" +cityName+ "&days=1&aqi=yes&alerts=yes";

        cityNameTV.setText(cityName);

        # Create a variabel RequestQueue
        RequestQueue requestQueue = Volley.newRequestQueue(MainActivity.this);

        # The weather API is a json object, so we have to make a json object request.
        JsonObjectRequest jsonObjectRequest = new JsonObjectRequest(Request.Method.GET, url, null, new Response.Listener<JSONObject>() {
            @Override
            public void onResponse(JSONObject response) {
                loadingPB.setVisibility(View.GONE);
                homeRL.setVisibility(View.VISIBLE);
                weatherRVModalArrayList.clear();

                # Take a close look at the syntax of accessing the data in Json format.
                try {
                    String temperature = response.getJSONObject("current").getString("temp_c");
                    temperatureTV.setText(temperature + "°c");
                    int isDay = response.getJSONObject("current").getInt("is_day");
                    String condition = response.getJSONObject("current").getJSONObject("condition").getString("text");
                    String conditionIcon = response.getJSONObject("current").getJSONObject("condition").getString("icon");
                    Picasso.get().load("http:".concat(conditionIcon)).into(iconIV);
                    conditionTV.setText(condition);

                    # Switch background colour
                    if (isDay==1) {
                        # Morning
                        Picasso.get().load("@assets/day light.jpg").into(backIV)
                    } else {
                        # Night
                        Picasso.get().load("@assets/night sky.jpg").into(backIV);
                    }

                    JSONObject forecastObj = response.getJSONObject("forecast");
                    JSONObject forecastArr = forecastObj.getJSONArray("forecastday").getJSONObject(0);
                    JSONArray hourArray = forecastArr.getJSONArray("hour");

                    for(int i=0; i < hourArray.length(); i++) {
                        JSONObject hourObj = hourArray.getJSONObject(i);

                        # Create variables to store the fetched data.
                        String time = hourObj.getString("time");
                        String temp_c = hourObj.getString("temp_c");
                        String icon = hourObj.getJSONObject("condition").getString("icon");
                        String wind_kph = hourObj.getString("wind_kph");
                        weatherRVModalArrayList.add(new WeatherRVModal(time, temp_c, icon, wind_kph));
                    }

                    # Notify adaptor about these variables.
                    weatherRVAdapter.notifyDataSetChanged();



                } catch (JSONException e) {
                    e.printStackTrace();
                }
            }
        }, new Response.ErrorListener() {
            @Override
            public void onErrorResponse(VolleyError error) {
                Toast.makeText(MainActivity.this, "Please enter valid city name", Toast.LENGTH_SHORT).show();
            }
        });


    }


## 4) Get city name

1) Create a method that gets the city name from the API latitude and longitude data.  
2) Initialise with "Not found" in case the method fails to get longitude & latitude values.

In [ ]:
    ...

    @Override
    protected void onCreate(Bundle savedInstanceState) {
        super.onCreate(savedInstanceState);

    ...

    # 1) Create a method
    private String getCityName(double longitude, double latitude){
        # 2) Initialise CityName.
        String cityName = "Not found";
        Geocoder gcd = new Geocoder(getBaseContext(), Locale.getDefault());
        try {
            List<Address> addresses = gcd.getFromLocation(latitude, longitude, 10);

            for (Address adr: addresses){
                if (adr!=null){                           # if the locality data is there,
                    String city = adr.getLocality();      # get that data.
                    if (city!=null && !city.equals("")){  # check if the locality data is correct.
                        cityName = city;                  # update cityName with that screened data.
                    } else {
                        Log.d("TAG", "CITY NOT FOUND");
                        Toast.makeText(this, "User City Not Found", Toast.LENGTH_SHORT).show();
                    }
                }
            }
        } catch (IOException e){
            e.printStackTrace();
        }

        return cityName;

    }

    ...

## 5) Get User Location

### 5-1) Configure Internet Permission in `AndroidManifest.xml`

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<manifest xmlns:android="http://schemas.android.com/apk/res/android"
    package="com.example.weatherapp" >

    # Configure Internet Permission to use the user's current location..
    <uses-permission android:name="android.permission.INTERNET"/>
    <uses-permission android:name="android.permission.ACCESS_NETWORK_STATE"/>
    <uses-permission android:name="android.permission.ACCESS_COARSE_LOCATION"/>
    <uses-permission android:name="android.permission.ACCESS_FINE_LOCATION"/>

    <application
    
    ...

</manifest>    

### 5-2) Variables - `LocationManager` & `PERMISSION_CODE`

In [ ]:
public class MainActivity extends AppCompatActivity {

    ...

    # User location-related variables
    # We need to ask for permission first when using user's current location.
    private LocationManager locationManager;
    private int PERMISSION_CODE = 1;

    ...

### 5-3) `LocationManager`


#### Inisialise and Check Permission
- 1) Inisialise the LocationManager.
- 2) Check whether the user has granted permission or not. If the permission is not granted, ask the user to grant the permission.

In [ ]:
    ...

    @Override
    protected void onCreate(Bundle savedInstanceState) {
        super.onCreate(savedInstanceState);

        ...

        # 1) Inisialise the LocationManager.
        locationManager = (LocationManager) getSystemService(Context.LOCATION_SERVICE);
        # 2) Permission check=
        if(ActivityCompat.checkSelfPermission(this, Manifest.permission.ACCESS_FINE_LOCATION)!= PackageManager.PERMISSION_GRANTED && ActivityCompat.checkSelfPermission(this,Manifest.permission.ACCESS_COARSE_LOCATION)!=PackageManager.PERMISSION_GRANTED) {
            ActivityCompat.requestPermissions(MainActivity.this, new String[]{Manifest.permission.ACCESS_FINE_LOCATION, Manifest.permission.ACCESS_COARSE_LOCATION}, PERMISSION_CODE);
        }

        ...

#### Get user's current location

#### Update city name and geolocation

In [ ]:
        ...

        # Once the permission is granted,
        Location location = locationManager.getLastKnownLocation(LocationManager.NETWORK_PROVIDER);
        # Get the user's city name using uer's current location obtained upon the granted permission.
        String cityName = getCityName(location.getLongitude(), location.getLatitude());
        # Pass the obtained cityName into getWeatherInfo to fetch the weather info.
        getWeatherInfo(cityName);

        searchIV.setOnClickListener(new View.OnClickListener(){
          @Override
          public void onClick(View v) {
              # Whatever user types on the city edit search bar (@id/idEditCity)
              # Turn the input into a string
              String city = cityEdt.getText().toString();

              # If no input, alert the user to enter something.
              if (city.isEmpty()){
                  Toast.makeText(MainActivity.this, "Please enter city name", Toast.LENGTH_SHORT).show();
              } else {
                  # if any input,
                  cityNameTV.setText(cityName);  # Update the city name display bar with the inputted text (city name)
                  getWeatherInfo(city);          # and fetch weather info.
              }
            }
        });
    }
    
    ...

#### `onRequestPermissionsResult`
- Handle the user response: 
  - if permissions granted: show granted status message. 
  - if denied: show denied status message and close the app.

In [ ]:
    ...
    
    # Once the permission is granted by the user,
    # You can call this built-in method automatically.
    @Override
    public void onRequestPermissionsResult(int requestCode, @NonNull String[] permissions, @NonNull int[] grantResults) {
        super.onRequestPermissionsResult(requestCode, permissions, grantResults);
        if (requestCode==PERMISSION_CODE) {
            if (grantResults.length > 0 && grantResults[0] == PackageManager.PERMISSION_GRANTED) {
                Toast.makeText(this, "Permissions granted", Toast.LENGTH_SHORT).show();
            } else {
                Toast.makeText(this, "Please provide permissions", Toast.LENGTH_SHORT).show();
                finish();
            }
        }
    }

    ...

## 6) Set App Icon


In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<manifest xmlns:android="http://schemas.android.com/apk/res/android"
    package="com.example.weatherapp" >

    <uses-permission android:name="android.permission.INTERNET"/>
    ...

    <application
        android:allowBackup="true"
        android:icon="@drawable/cloudy"         # set this to the icon of your choice.
        android:label="@string/app_name"
        android:roundIcon="@drawable/cloudy"    # set this to the icon of your choice.
        android:supportsRtl="true"
        android:theme="@style/Theme.WeatherApp" >

    ...

    </application>

</manifest>